In [1]:
import fridge
import fridge.utilities.h5Interface as h5i
import fridge.utilities.reactorInterface as RI
import os

from scipy.interpolate import interp1d as interp
from matplotlib import pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

import osbrain
from osbrain import run_nameserver
from osbrain import run_agent
from osbrain import logging
import numpy as np
import blackboard
import ka
import ka_rp as karp
import ka_br as kabr
import bb_sfr_opt
import time

# SFR Optimization

This example seeks to optimize a SFR design utilizing the blackboard framework.
We start first by taking a look at the SFR design data using the `h5` add `reactor` interfaces built into `FRDIGe` [cite].
For this we'll examine the objectives that will drive the optimization process: cycle length, plutonium mass, reactivity swing, and core average burnup.
Upon examination, we note that we have removed a significant portion of the design space by ensuring that the reactor design has a cycle length greater than 0 days.

In [4]:
h5_interface = h5i.h5Interface()
h5_interface.read_h5('/Users/ryanstewart/projects/sfr_database/SFR_DB.h5')

In [5]:
cycle_length_list = []
rx_swing_list = []
height = []
smear = []
pu_content = []
bu = []
pu_mass = []
for rx, core in h5_interface.h5file.items():
    try:
        rx_ = RI.reactorInterface(core)
        cycle_length = rx_.extrapolate_value('keff', 'time', 1.0)
        rx_swing = rx_.get_reactivity_swing(1.0, cycle_length)
        rx_swing_per_day = rx_swing/cycle_length
        avg_bu = rx_.get_assembly_avg(cycle_length, 'burnup')
        avg_bu_per_day = avg_bu/cycle_length
        pu = rx_.get_assembly_pu_mass() * 78 / 4
        if cycle_length > 0:
            height.append(rx_.rx['independent variables']['height'][0])
            smear.append(rx_.rx['independent variables']['smear'][0])
            pu_content.append(rx_.rx['independent variables']['pu_content'][0])
            cycle_length_list.append(cycle_length)
            rx_swing_list.append(rx_swing)
            bu.append(avg_bu)
            pu_mass.append(pu)
    except (KeyError, TypeError):
        print('didnt get {}'.format(core))

In [4]:
fig = px.scatter_3d(x=height, y=smear, z=cycle_length_list, color=pu_content, labels={'x':'Height (cm)', 'y': 'Smear', 'z':'Cycle Length (days)','color':'Pu Fraction'})
fig.show()
fig2 = px.scatter_3d(x=height, y=smear, z=rx_swing_list, color=pu_content, labels={'x':'Height (cm)', 'y': 'Smear', 'z':'Rx Swing (pcm)','color':'Pu Fraction'})
fig2.show()
fig3 = px.scatter_3d(x=height, y=smear, z=bu, color=pu_content, labels={'x':'Height (cm)', 'y': 'Smear', 'z':'Burnup (GWd/MTU)','color':'Pu Fraction'})
fig3.show()
fig4 = px.scatter_3d(x=height, y=smear, z=pu_mass, color=pu_content, labels={'x':'Height (cm)', 'y': 'Smear', 'z':'Pu Mass per Cycle (kg)','color':'Pu Fraction'})
fig4.show()

NameError: name 'height' is not defined

# Starting the Blackboard System

In [2]:
ns = run_nameserver()
bb = run_agent(name='bb', base=bb_sfr_opt.BbSfrOpt)
bb.set_attr(desired_results={'cycle length': (365, 1500), 
                             'reactivity swing': (0, 7500), 
                             'burnup': (0,175), 
                             'pu mass': (0, 1750)})
bb.generate_sm()

bb.connect_agent(karp.KaRpExplore, 'ka_rp_explore_1')
bb.connect_agent(karp.KaRpExplore, 'ka_rp_explore_2')
bb.connect_agent(karp.KaRpExploit, 'ka_rp_exploit')
bb.connect_agent(kabr.KaBr_lvl2, 'ka_br_lvl2')
bb.connect_agent(kabr.KaBr_lvl3, 'ka_br_lvl3')

Broadcast server running on 0.0.0.0:9091
NS running on 127.0.0.1:19474 (127.0.0.1)
URI = PYRO:Pyro.NameServer@127.0.0.1:19474
INFO [2020-05-04 20:54:01.205855] (bb): Connected agent ka_rp_explore_1 of agent type <class 'ka_rp.KaRpExplore'>
INFO [2020-05-04 20:54:01.259458] (bb): Connected agent ka_rp_explore_2 of agent type <class 'ka_rp.KaRpExplore'>
INFO [2020-05-04 20:54:01.315008] (bb): Connected agent ka_rp_exploit of agent type <class 'ka_rp.KaRpExploit'>
INFO [2020-05-04 20:54:01.364210] (bb): Connected agent ka_br_lvl2 of agent type <class 'ka_br.KaBr_lvl2'>
INFO [2020-05-04 20:54:01.415744] (bb): Connected agent ka_br_lvl3 of agent type <class 'ka_br.KaBr_lvl3'>
INFO [2020-05-04 20:54:04.437140] (bb): Selecting agent ka_rp_explore_1 (TV: 1.0) to execute (TE: 1)
INFO [2020-05-04 20:54:07.452013] (bb): Selecting agent ka_rp_explore_2 (TV: 1.0) to execute (TE: 2)
INFO [2020-05-04 20:54:10.463471] (bb): Selecting agent ka_br_lvl3 (TV: 10.0) to execute (TE: 3)
INFO [2020-05-04 20:5

INFO [2020-05-04 20:56:04.881472] (bb): Selecting agent ka_br_lvl2 (TV: 10.0) to execute (TE: 41)
INFO [2020-05-04 20:56:07.889923] (bb): Selecting agent ka_br_lvl3 (TV: 10.0) to execute (TE: 42)
INFO [2020-05-04 20:56:07.891350] (ka_br_lvl3): Executing agent ka_br_lvl3
INFO [2020-05-04 20:56:08.410627] (ka_br_lvl3):  Number of entries in 
 Level 3: 1, 24 
 Level 2: 1, 19 
 Level 1: 9, 3
INFO [2020-05-04 20:56:10.897793] (bb): Selecting agent ka_rp_exploit (TV: 2.0) to execute (TE: 43)
INFO [2020-05-04 20:56:13.915613] (bb): Selecting agent ka_rp_explore_1 (TV: 1.0) to execute (TE: 44)
INFO [2020-05-04 20:56:16.928769] (bb): Selecting agent ka_br_lvl3 (TV: 10.0) to execute (TE: 45)
INFO [2020-05-04 20:56:16.930802] (ka_br_lvl3): Executing agent ka_br_lvl3
INFO [2020-05-04 20:56:17.960167] (ka_br_lvl3):  Number of entries in 
 Level 3: 7, 25 
 Level 2: 1, 20 
 Level 1: 8, 4
INFO [2020-05-04 20:56:19.942940] (bb): Selecting agent ka_br_lvl3 (TV: 10.0) to execute (TE: 46)
INFO [2020-05-04

INFO [2020-05-04 20:58:08.361527] (ka_br_lvl3): Executing agent ka_br_lvl3
INFO [2020-05-04 20:58:08.897567] (ka_br_lvl3):  Number of entries in 
 Level 3: 6, 41 
 Level 2: 2, 34 
 Level 1: 13, 7
INFO [2020-05-04 20:58:11.369945] (bb): Selecting agent ka_br_lvl2 (TV: 10.0) to execute (TE: 83)
INFO [2020-05-04 20:58:14.382312] (bb): Selecting agent ka_br_lvl3 (TV: 10.0) to execute (TE: 84)
INFO [2020-05-04 20:58:14.383827] (ka_br_lvl3): Executing agent ka_br_lvl3
INFO [2020-05-04 20:58:14.918535] (ka_br_lvl3):  Number of entries in 
 Level 3: 5, 42 
 Level 2: 1, 35 
 Level 1: 14, 7
INFO [2020-05-04 20:58:17.395024] (bb): Selecting agent ka_br_lvl3 (TV: 10.0) to execute (TE: 85)
INFO [2020-05-04 20:58:17.397155] (ka_br_lvl3): Executing agent ka_br_lvl3
INFO [2020-05-04 20:58:17.929970] (ka_br_lvl3):  Number of entries in 
 Level 3: 4, 42 
 Level 2: 2, 35 
 Level 1: 14, 7
INFO [2020-05-04 20:58:20.407192] (bb): Selecting agent ka_br_lvl2 (TV: 10.0) to execute (TE: 86)
INFO [2020-05-04 20:

INFO [2020-05-04 21:00:06.367305] (ka_br_lvl3):  Number of entries in 
 Level 3: 4, 54 
 Level 2: 2, 46 
 Level 1: 16, 9
INFO [2020-05-04 21:00:08.838930] (bb): Selecting agent ka_br_lvl2 (TV: 10.0) to execute (TE: 122)
INFO [2020-05-04 21:00:11.853309] (bb): Selecting agent ka_br_lvl3 (TV: 10.0) to execute (TE: 123)
INFO [2020-05-04 21:00:11.855998] (ka_br_lvl3): Executing agent ka_br_lvl3
INFO [2020-05-04 21:00:12.396561] (ka_br_lvl3):  Number of entries in 
 Level 3: 3, 55 
 Level 2: 1, 47 
 Level 1: 16, 9
INFO [2020-05-04 21:00:14.867534] (bb): Selecting agent ka_br_lvl3 (TV: 10.0) to execute (TE: 124)
INFO [2020-05-04 21:00:14.869879] (ka_br_lvl3): Executing agent ka_br_lvl3
INFO [2020-05-04 21:00:15.408705] (ka_br_lvl3):  Number of entries in 
 Level 3: 2, 55 
 Level 2: 2, 47 
 Level 1: 16, 9
INFO [2020-05-04 21:00:17.877384] (bb): Selecting agent ka_br_lvl2 (TV: 10.0) to execute (TE: 125)
INFO [2020-05-04 21:00:20.886356] (bb): Selecting agent ka_br_lvl3 (TV: 10.0) to execute (T

In [3]:
while not bb.get_attr('_complete'):
    bb.publish_trigger()
    time.sleep(3)
    bb.controller()
    bb.send_executor()
    bb.wait_for_ka()
    bb.determine_complete()
    print('Finished trigger event: {}'.format(bb.get_attr('_trigger_event')))    

Finished trigger event: 1
Finished trigger event: 2
Finished trigger event: 3
Finished trigger event: 4
Finished trigger event: 5
Finished trigger event: 6
Finished trigger event: 7
Finished trigger event: 8
Finished trigger event: 9
Finished trigger event: 10
Finished trigger event: 11
Finished trigger event: 12
Finished trigger event: 13
Finished trigger event: 14
Finished trigger event: 15
Finished trigger event: 16
Finished trigger event: 17
Finished trigger event: 18
Finished trigger event: 19
Finished trigger event: 20
Finished trigger event: 21
Finished trigger event: 22
Finished trigger event: 23
Finished trigger event: 24
Finished trigger event: 25
Finished trigger event: 26
Finished trigger event: 27
Finished trigger event: 28
Finished trigger event: 29
Finished trigger event: 30
Finished trigger event: 31
Finished trigger event: 32
Finished trigger event: 33
Finished trigger event: 34
Finished trigger event: 35
Finished trigger event: 36
Finished trigger event: 37
Finished t

KeyboardInterrupt: 